In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
import tempfile
from matplotlib import pyplot as plt 
import numpy as np 
from keras.layers import Conv2D,Conv1D,Reshape,MaxPooling2D,Multiply,GlobalAveragePooling2D,Multiply,Lambda, Activation,GlobalMaxPooling2D,Dense,Input
from tensorflow.keras.applications.vgg19 import VGG19
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
import pickle
#from tensorflow.keras.layers.experimental import GroupNormalization

In [ ]:
seed_value = 42
tf.random.set_seed(seed_value)

In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow_addons as tfa
from tensorflow_addons.layers import GroupNormalization

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
#------------------------------------------------------------------------------
#  Loading CIFAR10 data
#------------------------------------------------------------------------------

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print("******************")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Convert class vectors to binary class matrices using one hot encoding
y_train_ohe = to_categorical(y_train, num_classes = 10)
y_test_ohe = to_categorical(y_test, num_classes = 10)

In [ ]:
print(type(x_train))

In [ ]:
# Data normalization
x_train=tf.image.resize(x_train,(48,48))
x_test=tf.image.resize(x_test,(48,48))
x_train = x_train.numpy()
x_test = x_test.numpy()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train  /= 255
x_test /= 255

print("******************")
print(x_train.shape)
print(y_train_ohe.shape)
print(x_test.shape) 
print(y_test_ohe.shape)

In [ ]:
x_val = x_train[40000:]
y_val = y_train_ohe[40000:]
print(x_val.shape)  
print(y_val.shape)

In [ ]:
x_train = x_train[:40000]
y_train_ohe = y_train_ohe[:40000]
print(x_train.shape)
print(y_train_ohe.shape)  

In [ ]:
vgg16_model = VGG16(weights='imagenet',
                    include_top=False, 
                    classes=10,
                    input_shape=(48,48,3)# input: images with 3 channels -> (224,224,3) tensors.
                   )

#Define the sequential model and add th VGG's layers to it
from tensorflow.keras.models import Sequential
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

from tensorflow.keras.layers import Dense, Flatten, Dropout
model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:

def flops_count(new_model):
    total_flops = 0
    for layer in new_model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            #print(f'{layer.name}') 
            kernel_size = layer.kernel_size
            input_channels = layer.input_shape[-1]
            output_channels = layer.output_shape[-1]
            height = layer.output_shape[1]
            width = layer.output_shape[2]
            flops = ((kernel_size[0] * kernel_size[1] * input_channels) * output_channels + output_channels) * height * width
            total_flops += flops
        elif isinstance(layer, tf.keras.layers.Dense):
            input_neurons = layer.input_shape[-1]
            output_neurons = layer.units
            flops = (input_neurons * output_neurons) + output_neurons
            total_flops += flops
    return total_flops
print(f'Model Flops: {flops_count(model)}')

In [ ]:
#------------------------------------------------------------------------------
# TRAINING THE CNN ON THE TRAIN/VALIDATION DATA
#------------------------------------------------------------------------------
from tensorflow.keras import optimizers
# initiate SGD optimizer
sgd = optimizers.SGD(lr=0.001, momentum=0.9)

# For a multi-class classification problem
model.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])


def lr_scheduler(epoch):
    return 0.001 * (0.1 ** (epoch // 40))
reduce_lr = LearningRateScheduler(lr_scheduler)

#mc = ModelCheckpoint('./weights.h5', monitor='val_accuracy', save_best_only=True, mode='max')
Monitor=[
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',mode='max',patience=10),
    tf.keras.callbacks.ModelCheckpoint('model.h5',monitor='val_accuracy',mode='max',save_best_only=True)
]

# initialize the number of epochs and batch size
EPOCHS = 100
BS = 32

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
    rotation_range=20, 
    zoom_range=0.15, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.15,
    horizontal_flip=True, 
    fill_mode="nearest")
 
# train the model
history = model.fit_generator(aug.flow(x_train,y_train_ohe, batch_size=BS,seed=42),validation_data=(x_val,y_val),
    steps_per_epoch=len(x_train) // BS,epochs=EPOCHS,callbacks=[reduce_lr,Monitor])

In [ ]:
model_loss, accuracy = model.evaluate(x_test, y_test_ohe, verbose=1)
print('Baseline test accuracy:', accuracy)

In [ ]:
# save the training weights for future use
#model.save_weights('cifar_100_model.ckpt')

In [ ]:
# save the training weights for future use
#model.save_weights('model_cifar10.ckpt')
model.save_weights('model.ckpt')

In [ ]:
#model.load_weights('model.ckpt')
#model.load_weights('model_cifar100_512.ckpt')
model.load_weights('model_cifar10.ckpt')

In [ ]:

!pip install -q tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

# Inference latency

In [ ]:
import time
# Prepare a sample input image for inference (adjust shape as needed)
input_image = x_test[0:1]  # Taking the first image for inference

# Warm-up the model (optional)
model.predict(input_image)

#input_image = x_test[0:100]

# Measure inference latency
n = 10000  # Number of repetitions
total_time = 0

for i in range(n):
    start_time = time.time()
    model.predict(x_test[i:i+1])
    end_time = time.time()
    total_time += (end_time - start_time)

average_latency = (total_time / n) *1000
print("Average Latency:", average_latency, "Milli seconds")
print("Total time:", total_time, " seconds")



# LP

In [ ]:
def AVGSTDGNFilterPruning(model):
    
    class GlobalStdPooling(tf.keras.layers.Layer):
        def call(self, inputs):
            # Calculate global average pooling
            mean = tf.reduce_mean(inputs, axis=[1, 2], keepdims=True)
            #max = tf.reduce_max(inputs, axis=[1, 2], keepdims=True) 
            #print('mean:')
            #tf.print(max)
            # Subtract the mean from each channel
            centered_inputs = inputs - mean

            # Square the centered differences
            squared_diff = tf.square(centered_inputs)
            #print("squared_diff:")
            #print(  squared_diff.shape)
            # Calculate the standard deviation
            std_dev = (tf.sqrt(tf.reduce_mean(squared_diff, axis=[1, 2], keepdims=True)+1e-5)) #epsilon=1e-5
            #print('std:')
            #tf.print(std_dev)
            # Flatten the std_dev tensor
            flattened_mean = tf.keras.layers.Flatten()( mean )
            channel_std = tf.keras.layers.Flatten()(std_dev)
            #tf.print(flattened_mean)
            #tf.print(flattened_mean-2 )

            return  channel_std
    
    
    def cbam(inputs, ratio=16):
        # Create ECA block
        x=inputs
        p = GlobalAveragePooling2D()(x) 
        q = GlobalStdPooling()(x) 
        x1=p+q
        #x1=Reshape((x.shape[-1],1))(x1)
        #x1 = Dense(units=x.shape[-1] // 16, activation='relu')(x1)
        #x1=Conv1D(filters=1,kernel_size=3,padding='same',activation='sigmoid')(x1)
        #x1 = Dense(units=x.shape[-1], activation='relu')(x1)
        #x1=Reshape((x.shape[-1],))(x1)
        x1=GroupNormalization(groups=4)(x1)

        x2 = GlobalMaxPooling2D()(x)
        #x2 = GlobalStdPooling()(x)
        #x2=m+n
        #x2=Reshape((x.shape[-1],1))(x2) 
        #x2=Conv1D(filters=1,kernel_size=3,padding='same',activation='sigmoid')(x2)
        #x2 = Dense(units=x.shape[-1] // 16, activation='relu')(x2)
        #x2 = Dense(units=x.shape[-1], activation='relu')(x2)
        #x2=Reshape((x.shape[-1],))(x2)
        #x2=GroupNormalization(groups=4)(x2)
        #x2 = GlobalStdPooling()(x)
        x2=GroupNormalization(groups=4)(x2)
        
        
        features=x1+x2
        features=Activation("sigmoid")(features)

        excitation = Reshape((1, 1, x.shape[-1]))(features)
        scale = Multiply()([x, excitation]) 
        return scale

    def build_vgg16_CBAM():

        # Create a new model using the functional API
        inputs = tf.keras.Input(shape=(48, 48, 3))
        x = inputs
        for layer in model.layers:       # Except 1st layer
            if isinstance(layer, layers.Conv2D):
                x = layer(x)
                x = cbam(x)  # Apply SE block after each Conv2D layer
            if isinstance(layer, MaxPooling2D):
                x = layer(x)

        # Classification layers
        x = layers.Flatten()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dense(512, activation='relu')(x)
        outputs = layers.Dense(10, activation='softmax')(x)  # CIFAR-10 has 10 classes

        new_model = tf.keras.Model(inputs, outputs)
        return new_model
    new_model = build_vgg16_CBAM()

    # Copy the weights layer by layer
    #for layer1, layer2 in zip(cbam_model.layers, new_model.layers):
       # weights = layer1.get_weights()
       # layer2.set_weights(weights)

    # copy Conv2D layer weight
    conv2D_index = [0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16]  # List of indices for Conv2D layers in the new model

    conv_index = 0  # Index variable to keep track of Conv2D layers in the new model
    for layer in new_model.layers:
        if isinstance(layer, Conv2D):
            model_layer = model.layers[conv2D_index[conv_index]]
            layer.set_weights(model_layer.get_weights())
            conv_index += 1

    # copy dense layer weight
    trained_flatten_index = None
    new_flatten_index = None

    for index, layer in enumerate(model.layers, start=0):
        if isinstance(layer, tf.keras.layers.Flatten):
            trained_flatten_index = index
            break

    for index, layer in enumerate(new_model.layers, start=0):
        if isinstance(layer, tf.keras.layers.Flatten):
            new_flatten_index = index
            break

    if trained_flatten_index is not None and new_flatten_index is not None:
        trained_dense_layers = model.layers[trained_flatten_index + 1:trained_flatten_index + 4]
        new_dense_layers = new_model.layers[new_flatten_index + 1:new_flatten_index + 4]

        for trained_dense_layer, new_dense_layer in zip(trained_dense_layers, new_dense_layers):
            new_dense_layer.set_weights(trained_dense_layer.get_weights())
    else:
        print("Flatten layer not found in one or both models.")

    # initiate SGD optimizer
    sgd = optimizers.SGD(lr=0.001, momentum=0.9)

    def lr_scheduler(epoch):
        return 0.001 * (0.5 ** (epoch // 15))
    reduce_lr = LearningRateScheduler(lr_scheduler)
    
    
    
    # new model retraining 
    new_model.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])
    new_model.summary()
    new_model.fit_generator(aug.flow(x_train,y_train_ohe, batch_size=BS),validation_data=(x_val,y_val),
    steps_per_epoch=len(x_train) // BS,epochs=50,callbacks=[reduce_lr])
    
    
    
    activation_average=[]
    for layer in new_model.layers:
        if isinstance(layer, tf.keras.layers.Reshape) and (isinstance(new_model.layers[new_model.layers.index(layer)+1],Multiply)):

            test_data=x_test[:1000] #only pick those sample which are correctly classified
            true_labels =y_test_ohe[:1000]
            predictions = new_model.predict(test_data)
            predicted_classes = np.argmax(predictions, axis=1)
            true_labels = np.argmax(true_labels, axis=1)
            correct_indices = np.where(predicted_classes == true_labels)[0]
            correct_test_data = test_data[correct_indices]

            # calculating activation for filter pruning

            inputs=new_model.input
            outputs=layer.output

            activation_model=tf.keras.Model(inputs=inputs,outputs=outputs) # create new block for activation calculation
            activation_SE=activation_model.predict(correct_test_data)

            # calculate the average activation from the SE block, reshape layer is used so we squeeze the shape

            acti_average=np.average(np.squeeze(activation_SE),axis=0)
            activation_average.append(acti_average)
            #activation_average

    #delete the model for memory clear up
    del new_model
    import gc
    gc.collect()
    
    return activation_average


In [ ]:
model.load_weights('model.ckpt')
#from tensorflow import keras

# Load the model from the .h5 file
#model = tf.keras.models.load_model('model.h5')

baseline_model_loss, baseline_model_accuracy = model.evaluate(
    x_test,y_test_ohe, verbose=1) 

print('Baseline test loss:  accuracy:', baseline_model_accuracy)
activation_average=[]
#activation calculate for filter importance
activation_average=AVGSTDGNFilterPruning(model)



In [ ]:
# Load the data from the file
import pickle
with open('activation_average_cifar10_layerpruned.pkl', "rb") as file:
    activation_average = pickle.load(file)

In [ ]:
# Calculate the percentage of values greater than 0.5 for each array
percentage_list = []

data_list=activation_average
for data in data_list:
    mean_val = np.mean(data)
    num_values_greater_than_0_5 = np.sum(data > 1.2*(mean_val))
    total_values = len(data)
    percentage = (num_values_greater_than_0_5 / total_values) * 100
    percentage_list.append(percentage)

# Resulting list of percentages
for percentage in percentage_list:
    print(f"Percentage of values over mean: {percentage:.2f}%")

In [ ]:
layer_pruned_index=np.argsort(layer_rank)[:7]
print(layer_pruned_index)
#layer_pruned_index=[7]

In [ ]:
layer_pruned_index=np.argsort(percentage_list)[:6]
print(layer_pruned_index)
#layer_pruned_index=[7]
layer_pruned_index=[ 9, 10, 11, 12,  8,  7,3]

In [ ]:
#vgg_model_conv_maxpool_index=[0,1,'M',2,3,'M',4,5,6,'M',7,8,9,'M',10,11,12]
layer_pruned_index=[ 9,  8, 10, 11,  7,  6, 12]

In [ ]:
model.load_weights('model.ckpt')
# Create a new model with  fewer layer
def create_new_layerpruned_model(model):
    new_layerpruned_model_input = Input(shape=(48, 48, 3))
    x =  new_layerpruned_model_input
    Conv2d_layer_count=0
    for layer in model.layers: 
        if isinstance(layer, Conv2D):
            
            if Conv2d_layer_count not in layer_pruned_index:   
                x = layer(x)
            Conv2d_layer_count+=1
            
        elif isinstance(layer, MaxPooling2D) :
            #x = layer(x)
            x=MaxPooling2D(2,2)(x) 
            
    x=Flatten()(x)
    x=Dense(512, activation='relu')(x)
    x=Dense(512, activation='relu')(x)
    x=Dense(10, activation='softmax')(x)
    new_layerpruned_model = tf.keras.Model(new_layerpruned_model_input, x)
    return new_layerpruned_model

new_layerpruned_model = create_new_layerpruned_model(model)
new_layerpruned_model.summary()

In [ ]:

#weight copy from old model to new layer pruned model
# we chosse to keep only the first 8 layer of the vgg16 model for histogram

Conv2d_layer_count=1
for layer,new_layer in zip(model.layers,new_layerpruned_model.layers[1:]):
    if isinstance(layer,Conv2D):
        new_layer.set_weights(layer.get_weights())
        Conv2d_layer_count+=1
        if(Conv2d_layer_count>6):
           break
new_layerpruned_model.layers[-1].set_weights(model.layers[-1].get_weights())
new_layerpruned_model.layers[-2].set_weights(model.layers[-2].get_weights())


# new model evaluation

# initiate SGD optimizer
sgd = optimizers.SGD(lr=0.001, momentum=0.9)

def lr_scheduler(epoch):
    return 0.001 * (0.5 ** (epoch // 50))
reduce_lr = LearningRateScheduler(lr_scheduler)

new_layerpruned_model.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])
     #Finetune

    
new_layerpruned_model.fit_generator(aug.flow(x_train,y_train_ohe, batch_size=BS),validation_data=(x_val,y_val),
    steps_per_epoch=len(x_train) // BS,epochs=1,callbacks=[reduce_lr])

baseline_model_loss, new_model_accuracy = new_layerpruned_model.evaluate(
    x_test,y_test_ohe, verbose=1) 

print('new test loss:  accuracy:', new_model_accuracy) 

print(f"Total number of parameters before pruning: {model.count_params()}")
print(f"Total number of parameters after pruning: {new_layerpruned_model.count_params()}")

print(f'Model Flops: {flops_count(model)}')

print(f'Layer pruned Model Flops: {flops_count(new_layerpruned_model)}')

print(f'Flops reduction:{(1-(flops_count(new_layerpruned_model)/flops_count(model)))*100}%')
print(f'Parameters reduction:{(1-(new_layerpruned_model.count_params()/model.count_params()))*100}%')

In [ ]:
#print(f'Model Flops: {flops_count(model)}')
def flops_count(new_model):
    total_flops = 0
    for layer in new_model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            #print(f'{layer.name}') 
            kernel_size = layer.kernel_size
            input_channels = layer.input_shape[-1]
            output_channels = layer.output_shape[-1]
            height = layer.output_shape[1]
            width = layer.output_shape[2]
            flops = ((kernel_size[0] * kernel_size[1] * input_channels) * output_channels + output_channels) * height * width
            total_flops += flops
        elif isinstance(layer, tf.keras.layers.Dense):
            input_neurons = layer.input_shape[-1]
            output_neurons = layer.units
            flops = (input_neurons * output_neurons) + output_neurons
            total_flops += flops
    return total_flops

#  and assign weight from base layer by considering activation average where wheight shape does not match

In [ ]:
# Load the data from the file
with open('activation_average_cifar10_layerpruned.pkl', "rb") as file:
    activation_average = pickle.load(file)

In [ ]:
# Calculate the percentage of values greater than 0.5 for each array
percentage_list = []

data_list=activation_average
for data in data_list:
    mean_val = np.mean(data)
    num_values_greater_than_0_5 = np.sum(data > (1.2*mean_val))
    total_values = len(data)
    percentage = (num_values_greater_than_0_5 / total_values) * 100
    percentage_list.append(percentage)

# Resulting list of percentages
for percentage in percentage_list:
    print(f"Percentage of values over mean: {percentage:.2f}%")

In [ ]:
layer_pruned_index=np.argsort(percentage_list)[:13]
#layer_pruned_index=np.argsort(percentage_list)[::-1][:6]
print(layer_pruned_index)
#layer_pruned_index=[7]
#[ 9  8 10 11  7  6 12 5]
 6  5  4  0  1  2  3

In [ ]:
#layer_pruned_index=[ 9 ,8, 10,11,  7,  12]

In [ ]:
# Define the dictionary with keys and values
keys = [0, 1, 'M1', 2, 3, 'M2', 4, 5, 6, 'M3', 7, 8, 9, 'M4', 10, 11, 12,'M5']
values = [64, 64, 0, 128, 128, 0, 256, 256, 256, 0, 512, 512, 512, 0, 512, 512, 512,0]

result_dict = {}

# Define the list of indices to set to 0
#layer_pruned_index = [9, 3, 10, 11, 7, 6]

# Populate the dictionary with key-value pairs
for key, value in zip(keys, values):
    if key in layer_pruned_index:
        result_dict[key] = [value, 0]  # Set to [value, 0] if in layer_pruned_index
    else:
        result_dict[key] = [value, 1]  # Set to [value, value] if not in layer_pruned_index

# Print the updated dictionary
print(result_dict)

In [ ]:
inputs= Input(shape=(48, 48, 3))
x=inputs
maxpool_list=['M1','M2','M3','M4','M5']
for key, value in zip(keys, values):
  if result_dict[key][1]==1:
    if key  not in maxpool_list:
      print(result_dict[key][0])
      x=Conv2D(result_dict[key][0],kernel_size=(3,3),padding='same',activation='relu')(x)
    if key   in maxpool_list:
      x=MaxPooling2D(2,2)(x)

x=Flatten()(x)
x=Dense(512, activation='relu')(x)
x=Dense(512, activation='relu')(x)
x=Dense(100, activation='softmax')(x)
new_layerpruned_model = tf.keras.Model(inputs, x)

new_layerpruned_model.summary()

In [ ]:
model.load_weights('model.ckpt')


conv_layer_kept_index=np.sort(np.argsort(percentage_list)[6:])
#conv_layer_kept_index=[0,1,2,3,4,5,6]
print(conv_layer_kept_index)

layer_keeping_index=0
model_conv_count=0
#enter to the base model to check the kept layer for new model
for layer in model.layers:
   if isinstance(layer, Conv2D):
      if layer_keeping_index in conv_layer_kept_index:
         desired_layer=0
        # enter the new model to copy the base model weight to new model if layer match in both model
         for new_layer in new_layerpruned_model.layers[1:]:
            if isinstance(new_layer, Conv2D):
            
               if model_conv_count==desired_layer:  # check where to assign 
                  index = np.where(conv_layer_kept_index == layer_keeping_index)[0]
                 #in_dex=conv_layer_kept_index.index(layer_keeping_index)-conv_layer_kept_index[index-1]
                 #print(index-1)
                  if conv_layer_kept_index[index]==0 or conv_layer_kept_index[index]-conv_layer_kept_index[index-1]<=1: # check consecutive layer or not
                     #print(layer.get_weights()) 
                     print(f'No shape change conv layer: {layer_keeping_index}')
                     #print(new_layer.get_weights()) 
                     print('.............................')
                     
                     new_layer.set_weights(layer.get_weights())
                     #print(new_layer.get_weights()) 
                     break
                    
                  else:
                    # Sort the importance list and get the indices in descending order
                     sorted_indices = np.argsort(activation_average[layer_keeping_index])[::-1]
                     print(f' Shape change conv layer: {layer_keeping_index}')
                    #print(sorted_indices)
                     #print(activation_average[layer_keeping_index])
                    # Get the weights from the original Conv2D layer
                     original_weights = layer.get_weights()
                    # Create a new weight matrix with the previous layer  filters by considering activation average
                     #print(result_dict[int(conv_layer_kept_index[index-1])][0])
                        # only weight depth change to bias
                        
                     if conv_layer_kept_index[0]!=0 and model_conv_count==0 :# if first layer pruned
                        new_depth=3
                     else:
                        new_depth= result_dict[int(conv_layer_kept_index[index-1])][0]  # remain depth 
                     print(f'new_depth:{new_depth}')
                     print(sorted_indices[:new_depth])
                     if new_depth== original_weights[0].shape[2]: # check for same depth ie. layer 5 and 7
                        new_layer.set_weights(original_weights)
                     else:
                        new_weights  = [original_weights[0][:, :,sorted_indices[:new_depth],: ], original_weights[1]]
                     # Inspect the shape of the weight arrays
                        
                     #for i, weight in enumerate(new_weights):
                        #print(f"Weight {i+1} shape: {weight.shape}")
                        new_layer.set_weights(new_weights)
                     break
               desired_layer=desired_layer+1
        
        
         model_conv_count=model_conv_count+1

      layer_keeping_index=layer_keeping_index+1

new_layerpruned_model.layers[-1].set_weights(model.layers[-1].get_weights())
new_layerpruned_model.layers[-2].set_weights(model.layers[-2].get_weights())
#new_layerpruned_model.layers[-3].set_weights(model.layers[-3].get_weights())

In [ ]:
def flops_count(new_model):
    total_flops = 0
    for layer in new_model.layers:
        if isinstance(layer, tf.keras.layers.Conv2D):
            #print(f'{layer.name}') 
            kernel_size = layer.kernel_size
            input_channels = layer.input_shape[-1]
            output_channels = layer.output_shape[-1]
            height = layer.output_shape[1]
            width = layer.output_shape[2]
            flops = ((kernel_size[0] * kernel_size[1] * input_channels) * output_channels + output_channels) * height * width
            total_flops += flops
        elif isinstance(layer, tf.keras.layers.Dense):
            input_neurons = layer.input_shape[-1]
            output_neurons = layer.units
            flops = (input_neurons * output_neurons) + output_neurons
            total_flops += flops
    return total_flops

In [ ]:
# new model evaluation

# initiate SGD optimizer
sgd = optimizers.SGD(lr=0.001, momentum=0.9)

def lr_scheduler(epoch):
    return 0.001 * (0.5 ** (epoch // 50))
reduce_lr = LearningRateScheduler(lr_scheduler)

new_layerpruned_model.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])
     #Finetune

    
new_layerpruned_model.fit_generator(aug.flow(x_train,y_train_ohe, batch_size=BS),validation_data=(x_val,y_val),
    steps_per_epoch=len(x_train) // BS,epochs=205,callbacks=[reduce_lr])

baseline_model_loss, new_model_accuracy = new_layerpruned_model.evaluate(
    x_test,y_test_ohe, verbose=1) 

print('new test loss:  accuracy:', new_model_accuracy) 

print(f"Total number of parameters before pruning: {model.count_params()}")
print(f"Total number of parameters after pruning: {new_layerpruned_model.count_params()}")

print(f'Model Flops: {flops_count(model)}')

print(f'Layer pruned Model Flops: {flops_count(new_layerpruned_model)}')

print(f'Flops reduction:{(1-(flops_count(new_layerpruned_model)/flops_count(model)))*100}%')
print(f'Parameters reduction:{(1-(new_layerpruned_model.count_params()/model.count_params()))*100}%')

# Inference latency

In [ ]:
import time
# Prepare a sample input image for inference (adjust shape as needed)
input_image = x_test[0:1]  # Taking the first image for inference

# Warm-up the model (optional)
new_layerpruned_model.predict(input_image)

#input_image = x_test[0:100]

# Measure inference latency
n = 10000  # Number of repetitions
total_time = 0

for i in range(n):
    start_time = time.time()
    new_layerpruned_model.predict(x_test[i:i+1])
    end_time = time.time()
    total_time += (end_time - start_time)

average_latency = (total_time / n) *1000
print("Average Latency:", average_latency, "Milli seconds")
print("Total Latency:", total_time, "seconds")


# Linear Classifier Probe

In [ ]:
Linear_classifier_probe_accuracy=[]
for linear_probe_position in range(13):
    inputs= Input(shape=(48, 48, 3))
    x=inputs
    count_number_of_conv_in_main_model=0
    for layer in model.layers:
        if isinstance(layer, MaxPooling2D) :
            #x = layer(x)
            x=MaxPooling2D(2,2)(x) 
        elif isinstance(layer,  Conv2D):
            x=layer(x)
            if linear_probe_position==count_number_of_conv_in_main_model:
                break
            count_number_of_conv_in_main_model=count_number_of_conv_in_main_model+1
            
        
            
    x=Flatten()(x)
    x=Dense(512, activation='relu')(x)
    x=Dense(512, activation='relu')(x)
    x=Dense(100, activation='softmax')(x)
    new_linearprobe_model = tf.keras.Model(inputs, x)

    new_linearprobe_model.summary()
    
    #wweight copy
    
    Conv2d_layer_count=0
    for layer,new_layer in zip(model.layers,new_linearprobe_model.layers[1:]):
        if isinstance(layer,Conv2D):
            new_layer.set_weights(layer.get_weights())
            Conv2d_layer_count+=1
            if(Conv2d_layer_count>linear_probe_position):
               break
    
    # Determine the index of the layer where you want to start training the last two layers
    # For example, if you want to freeze the layers up to the last three layers, use len(model.layers) - 3 as the start_index
    start_index = len(new_linearprobe_model.layers) - 3  # Start training from the last two layers

    # Freeze layers up to the specified index
    for layer in new_linearprobe_model.layers[:start_index]:
        layer.trainable = False
        
    
        # new model evaluation

    # initiate SGD optimizer
    sgd = optimizers.SGD(lr=0.001, momentum=0.9)

    def lr_scheduler(epoch):
        return 0.001 * (0.5 ** (epoch // 20))
    reduce_lr = LearningRateScheduler(lr_scheduler)
    Monitor=[
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',mode='max',patience=10),
    #tf.keras.callbacks.ModelCheckpoint('model.h5',monitor='val_accuracy',mode='max',save_best_only=True)
     ]

    new_linearprobe_model.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])
         #Finetune


    new_linearprobe_model.fit_generator(aug.flow(x_train,y_train_ohe, batch_size=BS),validation_data=(x_val,y_val),
        steps_per_epoch=len(x_train) // BS,epochs=100,callbacks=[reduce_lr,Monitor])

    baseline_model_loss, new_model_accuracy = new_linearprobe_model.evaluate(
        x_test,y_test_ohe, verbose=1) 

    print(f'Linear classifier probe {linear_probe_position+1} test accuracy:', new_model_accuracy)
    
    Linear_classifier_probe_accuracy.append(new_model_accuracy)
    print(Linear_classifier_probe_accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Sample data (replace this with your list of values)
values = [10, 25, 5, 30, 15]

# Create bar chart
plt.bar(range(1, len(Linear_classifier_probe_accuracy) + 1), Linear_classifier_probe_accuracy)  # Start from 1 and end at len(values)

# Add labels to the x-axis (numeric values 1, 2, 3, ...)
plt.xlabel("Layer")
plt.xticks(range(1, len(Linear_classifier_probe_accuracy) + 1))

# Add a label to the y-axis (optional)
plt.ylabel("Accuracy")

# Add a title (optional)
plt.title("VGG-16")

# Show the bar chart
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Sample data (replace this with your list of values)
values = [10, 25, 5, 30, 15]

# Create bar chart
plt.bar(range(1, len(Linear_classifier_probe_accuracy) + 1), Linear_classifier_probe_accuracy)  # Start from 1 and end at len(values)

# Add labels to the x-axis (numeric values 1, 2, 3, ...)
plt.xlabel("Layer")
plt.xticks(range(1, len(Linear_classifier_probe_accuracy) + 1))

# Add a label to the y-axis (optional)
plt.ylabel("Accuracy")

# Add a title (optional)
plt.title("VGG-16 ")

# Show the bar chart
plt.show()